### References
https://github.com/vanvalenlab/deepcell-tf/blob/master/scripts/feature_pyramids/RetinaNet%20-%20Movie.ipynb

In [1]:
import os
import datetime
import errno
import argparse

import numpy as np

import deepcell

# Load data

In [2]:
from deepcell.utils.data_utils import get_data
from deepcell.utils.tracking_utils import load_trks

DATA_DIR = '/data/training_data/cells/3T3/NIH/movie'
DATA_FILE = os.path.join(DATA_DIR, 'nuclear_movie_3T3_0-2_same.trks')

# Load Information for hardcoded image size training
seed = 1
test_size = .2
train_dict, test_dict = get_data(DATA_FILE, mode='siamese_daughters', seed=seed, test_size=test_size)
X_train, y_train = train_dict['X'], train_dict['y']
X_test, y_test = test_dict['X'], test_dict['y']

print(' -\nX.shape: {}\ny.shape: {}'.format(train_dict['X'].shape, train_dict['y'].shape))

 -
X.shape: (192, 30, 154, 182, 1)
y.shape: (192, 30, 154, 182, 1)


# File Contants

In [3]:
# Set up other required filepaths
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)
ROOT_DIR = '/data' # mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# Model Parameters

In [7]:
# Each head of the model uses its own loss
from deepcell.losses import RetinaNetLosses

sigma = 3.0
alpha = 0.25
gamma = 2.0
iou_threshold = 0.5
max_detections = 100
mask_size = (28, 28)

retinanet_losses = RetinaNetLosses(
    sigma=sigma, alpha=alpha, gamma=gamma,
    iou_threshold=iou_threshold,
    mask_size=mask_size)

loss = {
    'regression': retinanet_losses.regress_loss,
    'classification': retinanet_losses.classification_loss,
}

# Create RetinaMask Model

In [8]:
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler

model_name = 'trackrcnn_model'
backbone = 'resnet50'  # vgg16, vgg19, resnet50, densenet121, densenet169, densenet201

n_epoch = 10  # Number of training epochs
lr = 1e-5

optimizer = Adam(lr=lr, clipnorm=0.001)

lr_sched = rate_scheduler(lr=lr, decay=0.99)

batch_size = 1

num_classes = 1  # "object" is the only class

In [9]:
from deepcell.utils.retinanet_anchor_utils import get_anchor_parameters

flat_shape = [y_train.shape[0] * y_train.shape[1]] + list(y_train.shape[2:])
flat_y = np.reshape(y_train, tuple(flat_shape)).astype('int')

# Generate backbone information from the data
backbone_levels, pyramid_levels, anchor_params = get_anchor_parameters(flat_y)

fpb = 5  # number of frames in each training batch

In [11]:
from deepcell import model_zoo

# Pass frames_per_batch > 1 to enable 3D mode!
model = model_zoo.RetinaNet(
    backbone=backbone,
    use_imagenet=True,
    panoptic=False,
    frames_per_batch=fpb,
    num_classes=num_classes,
    input_shape=X_train.shape[2:],
    backbone_levels=backbone_levels,
    pyramid_levels=pyramid_levels,
    num_anchors=anchor_params.num_anchors())

prediction_model = model_zoo.retinanet_bbox(
    model,
    panoptic=False,
    frames_per_batch=fpb,
    max_detections=100,
    anchor_params=anchor_params)

prediction_model.summary()

Model: "retinanet-bbox"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        [(None, 5, 154, 182, 0                                            
__________________________________________________________________________________________________
time_distributed_30 (TimeDistri (None, 5, 154, 182,  0           image_input[0][0]                
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, 5, 154, 182,  6           time_distributed_30[0][0]        
__________________________________________________________________________________________________
time_distributed_33 (TimeDistri (None, 5, 39, 46, 25 229760      time_distributed_31[0][0]        
_____________________________________________________________________________________

# Train RetinaMask Model

### Training Parameters

In [13]:

from deepcell.image_generators import RetinaMovieDataGenerator

datagen = RetinaMovieDataGenerator(
    rotation_range=180,
    zoom_range=(0.8, 1.2),
    horizontal_flip=True,
    vertical_flip=True)

datagen_val = RetinaMovieDataGenerator()

In [14]:
train_data = datagen.flow(
    {'X': X_train, 'y': y_train},
    batch_size=1,
    include_masks=False,
    include_final_detection_layer=False,
    frames_per_batch=fpb,
    pyramid_levels=pyramid_levels,
    anchor_params=anchor_params)

val_data = datagen_val.flow(
    {'X': X_test, 'y': y_test},
    batch_size=1,
    include_masks=False,
    include_final_detection_layer=False,
    frames_per_batch=fpb,
    pyramid_levels=pyramid_levels,
    anchor_params=anchor_params)

W0120 06:00:29.835370 140716510050112 retinanet.py:633] Removing 2 of 192 images with fewer than 3 objects.
W0120 06:00:31.296959 140716510050112 retinanet.py:633] Removing 1 of 48 images with fewer than 3 objects.


In [15]:
from tensorflow.keras import callbacks
from deepcell.callbacks import RedirectModel, Evaluate

iou_threshold = 0.5
score_threshold = 0.01
max_detections = 100

model.fit_generator(
    train_data,
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=n_epoch,
    validation_data=val_data,
    validation_steps=X_test.shape[0] // batch_size,
    callbacks=[
        callbacks.LearningRateScheduler(lr_sched),
        callbacks.ModelCheckpoint(
            os.path.join(MODEL_DIR, model_name + '.h5'),
            monitor='val_loss',
            verbose=1,
            save_best_only=True,
            save_weights_only=False),
        RedirectModel(
            Evaluate(val_data,
                     iou_threshold=iou_threshold,
                     score_threshold=score_threshold,
                     max_detections=max_detections,
                     frames_per_batch=fpb,
                     weighted_average=True),
            prediction_model)
    ])

Epoch 1/10
191/192 [============================>.] - ETA: 0s - loss: 3.0883 - regression_loss: 2.5309 - classification_loss: 0.5574
Epoch 00001: val_loss improved from inf to 2.53996, saving model to /data/models/retinamovie_model.h5
525 instances of class 0 with average precision: 0.5584
mAP: 0.5584
192/192 [==============================] - 201s 1s/step - loss: 3.0847 - regression_loss: 2.5285 - classification_loss: 0.5562 - val_loss: 2.5400 - val_regression_loss: 2.1741 - val_classification_loss: 0.3658
Epoch 2/10
191/192 [============================>.] - ETA: 0s - loss: 2.2182 - regression_loss: 1.9423 - classification_loss: 0.2759
Epoch 00002: val_loss improved from 2.53996 to 1.98636, saving model to /data/models/retinamovie_model.h5
525 instances of class 0 with average precision: 0.8220
mAP: 0.8220
192/192 [==============================] - 147s 764ms/step - loss: 2.2169 - regression_loss: 1.9414 - classification_loss: 0.2755 - val_loss: 1.9864 - val_regression_loss: 1.7613 -